In [25]:
# from ipyfilechooser import FileChooser
from netCDF4 import Dataset
import numpy.ma as ma
import numpy as np
from ipyleaflet import Polyline, Map, Marker, basemap_to_tiles, basemaps, AwesomeIcon

In [26]:
# Select Argo file
# projDir = 'Argo_data/'
# fc = FileChooser(projDir)
# display(fc)

In [27]:
# argo_fn = fc.selected
argo_fn = 'Argo_data//2902730_prof.nc'

In [36]:
# Open NetCDF Argo file
nc_fid = Dataset(argo_fn,'r')
vars = nc_fid.variables.keys()
# display(vars)

In [29]:
lat = nc_fid.variables['LATITUDE'][:]
lon = nc_fid.variables['LONGITUDE'][:]
dateJD = nc_fid.variables['JULD'][:]
date_ref = nc_fid.variables['REFERENCE_DATE_TIME'][:]
nc_fid.close()

In [30]:
# Reference datetime
dd=date_ref.tobytes()
dd=dd.decode('utf-8')
ddnp = np.datetime64(dd[0:4]+'-'+dd[4:6]+'-'+dd[6:8]+'T'+dd[8:10]+':'+dd[10:12]+':'+dd[12:14])
# print(ddnp)

1950-01-01T00:00:00


In [31]:
date = ddnp + np.array(dateJD//1,dtype='timedelta64[D]') + np.array((dateJD%1)*24*60*60,dtype='timedelta64[s]')
# date

In [32]:
nProf = lat.count()
# nProf

In [37]:
latitude = lat.reshape(nProf,1)
longitude = lon.reshape(nProf,1)
coords = ma.concatenate([latitude, longitude],axis=1)
display(coords)

masked_array(
  data=[[ 12.003, 130.016],
        [ 12.836, 130.533],
        [ 13.351, 131.222],
        [ 12.74 , 131.426],
        [ 12.169, 131.459],
        [ 11.513, 131.998],
        [ 11.433, 132.603],
        [ 11.499, 132.688],
        [ 11.539, 132.585],
        [ 11.552, 132.071],
        [ 11.859, 131.436],
        [ 11.975, 130.887],
        [ 12.111, 130.151],
        [ 12.208, 130.061],
        [ 11.927, 130.025],
        [ 12.156, 129.157],
        [ 12.392, 128.87 ],
        [ 12.33 , 128.865],
        [ 12.038, 128.611],
        [ 11.942, 128.031],
        [ 11.532, 127.047],
        [ 11.064, 126.649],
        [ 10.06 , 126.644],
        [  8.907, 126.875],
        [  9.705, 127.115],
        [  9.325, 126.828],
        [  9.712, 127.297],
        [  9.782, 127.32 ],
        [ 10.216, 127.806],
        [ 10.604, 127.654],
        [ 10.669, 127.129],
        [ 10.581, 127.025],
        [ 10.85 , 127.02 ],
        [ 11.   , 126.811],
        [ 10.39 , 126.569],
      

In [34]:
argoTraj_line = Polyline(
    locations=coords.tolist(),
    color="black" ,
    fill=False,
    weight=3
)

In [35]:
from ipywidgets import Label
label = Label()
display(label)
def handle_interaction(**kwargs):
    if kwargs.get('type') == 'mousemove':
        m_coords = kwargs.get('coordinates')
        m_coords[0] = round(m_coords[0],5)
        m_coords[1] = round(m_coords[1],5)
        label.value = str(m_coords[0])+'  '+str(m_coords[1])

#
center = (lat.mean(), lon.mean())
m = Map(basemap=basemap_to_tiles(basemaps.Esri.NatGeoWorldMap), center=center, zoom=5)
m.add_layer(argoTraj_line);
for i in range(0,nProf):
    marker = Marker(location = coords[i].tolist(),draggable=False)
    marker.title = 'Profile '+str(i+1)+'\n'+str(date[i])
    marker.rise_on_hover = True
    marker.icon = AwesomeIcon(name='share',marker_color='beige',icon_color='black',spin=False)
    m.add_layer(marker)
m.layout.width = '100%'
m.layout.height = '500px'
m.on_interaction(handle_interaction)
m

Label(value='')

Map(center=[12.967607843137257, 127.63063725490196], controls=(ZoomControl(options=['position', 'zoom_in_text'…